<div style="border: 2px solid #8A9AD0; margin: 1em 0.2em; padding: 0.5em;">

# Data Manipulation Olympics - SQL

by [Saskia Hiltemann](https://training.galaxyproject.org/hall-of-fame/shiltemann/), [Helena Rasche](https://training.galaxyproject.org/hall-of-fame/hexylena/)

CC-BY licensed content from the [Galaxy Training Network](https://training.galaxyproject.org/)

**Objectives**

- How can I do basic data manipulation in SQL?
- Which functions are available to convert, reformat, filter, sort etc my data stored in a database?

**Objectives**

- Familiarize yourself with data manipulation in SQL
- Perform basic SQL query tasks in Galaxy
- Reason about the expected outcome of tools

**Time Estimation: 1h**
</div>


<!--
Note to contributors: feel free to add sections here to include additional data manipulation options.
Make sure each section is independent of each other, i.e. each section should start with the olympics.tsv file.
Also make sure to include many exercises (with answers) for your section!
-->
<h1 id="introduction">Introduction</h1>
<p>Scientific analyses often consist of a number of tools that run one after the other, in order to go from the raw data to scientific insight. Between these specialized tools, simple data manipulation steps are often needed as a kind of “glue” between tools. For example, the output of tool A may produce a file that contains all the information needed as input for tool B, but tool B expects the columns in a different order. Or in genomic data analysis, some tools expect chromosome X to be listed as <code style="color: inherit">chrX</code>, while others simply expect <code style="color: inherit">X</code>. In these situations, extra data manipulation steps are needed to prepare files for input to analysis tools.</p>
<p>Galaxy has a large collection of tools to perform such basic data manipulation tasks, and becoming familiar with these operations will allow to perform your analysis more easily in Galaxy (and outside).</p>
<blockquote class="agenda" style="border: 2px solid #86D486;display: none; margin: 1em 0.2em">
<div class="box-title agenda-title" id="agenda">Agenda</div>
<p>In this tutorial, we will cover:</p>
<ol id="markdown-toc">
<li><a href="#introduction" id="markdown-toc-introduction">Introduction</a></li>
<li><a href="#cheatsheet" id="markdown-toc-cheatsheet">Cheatsheet</a></li>
<li><a href="#background" id="markdown-toc-background">Background</a></li>
<li><a href="#preamble" id="markdown-toc-preamble">Preamble</a></li>
</ol>
</blockquote>
<h1 id="cheatsheet">Cheatsheet</h1>
<p>Here is an overview table of the different data manipulations in this tutorial, with links to the tools in Galaxy.</p>
<table>
<thead>
<tr>
<th>Operation</th>
<th>Description</th>
<th>Galaxy Tool</th>
</tr>
</thead>
<tbody>
<tr>
<td>Compute on rows</td>
<td>to derive new column values from existing ones</td>
<td><code style="color: inherit">SELECT x * 2 FROM y</code></td>
</tr>
<tr>
<td>Concatenate datasets</td>
<td>one after the other</td>
<td><code style="color: inherit">SELECT * FROM x; union all; SELECT * FROM y</code></td>
</tr>
<tr>
<td>Counting</td>
<td>Count occurrences of values in a column</td>
<td><code style="color: inherit">SELECT count(x) FROM y where x = 'value'</code></td>
</tr>
<tr>
<td>Cut Columns</td>
<td>By header name</td>
<td><code style="color: inherit">SELECT x, y, z FROM a</code></td>
</tr>
<tr>
<td>Filter</td>
<td>Remove rows based on values in one or more columns</td>
<td><code style="color: inherit">... WHERE x = 'value'</code></td>
</tr>
<tr>
<td>Find and Replace</td>
<td>in a specific column</td>
<td><code style="color: inherit">REPLACE()</code>, <code style="color: inherit">regexp_replace</code> in postgresql</td>
</tr>
<tr>
<td>Group on a column</td>
<td>And perform simple operations (count, mean, min, max etc)</td>
<td><code style="color: inherit">... GROUP BY x ...</code></td>
</tr>
<tr>
<td>Join two Datasets</td>
<td>side by side on a specified field</td>
<td><code style="color: inherit">SELECT * FROM x, y JOIN x.id = y.id</code></td>
</tr>
<tr>
<td>Select First lines</td>
<td>Good for finding top 10s or saving header lines</td>
<td><code style="color: inherit">... LIMIT 10</code></td>
</tr>
<tr>
<td>Sort on a column</td>
<td>Change the order of the rows based on values in one or more columns</td>
<td><code style="color: inherit">... ORDER BY x ASC</code></td>
</tr>
<tr>
<td>Unique</td>
<td>Remove duplicate rows</td>
<td><code style="color: inherit">SELECT DISTINCT x FROM y</code></td>
</tr>
</tbody>
</table>
<p>In this tutorial, these functions are explained in more detail, and we provide some exercises for you to practice.</p>
<h1 id="background">Background</h1>
<p>In this tutorial, we will use as our dataset a table with results from the Olympics, from the games in Athens in 1896 until Tokyo in 2020. The objective is to familiarize you with a large number of the most important data manipulation tools in Galaxy. Much like the Olympics, there are many different disciplines (types of operations), and for each operation there are often multiple techniques (tools) available to athletes (data analysts, you) that are great for achieving the goal.</p>
<p><img src="{% link topics/introduction/tutorials/data-manipulation-olympics/images/cover.jpg %}" alt="image of olympic rings, logo and two athletes around the words &quot;Data Analysis Olympics&quot;. " width="320" height="180" loading="lazy" /></p>
<p>We will show you many of these commonly needed data manipulation operations, and some examples of how to perform them in Galaxy. We also provide many exercises so that you can train your skills and become a data manipulation Olympian!</p>
<h1 id="preamble">Preamble</h1>


In [ ]:
# This preamble sets up the sql "magic" for jupyter. Use %%sql in your cells to write sql!
!python3 -m pip install ipython-sql sqlalchemy
!wget -c https://zenodo.org/record/6803028/files/olympics.db
import sqlalchemy
engine = sqlalchemy.create_engine("sqlite:///olympics.db")
%load_ext sql
%sql sqlite:///olympics.db
%config SqlMagic.displaycon=False

<h1 id="download-data">Download Data</h1>
<p>Before we can do any manipulation, we will need some data. Let’s download our table with Olympics results now.</p>


In [ ]:
%%sql
SELECT * FROM olympics LIMIT 10;

<p>And now we can start querying the database:</p>


In [ ]:
%%sql
SELECT
    name
FROM
    sqlite_schema

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<ol>
<li>What tables are available?</li>
<li>How are they structured?</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution"><button class="gtn-boxify-button solution" type="button" aria-controls="solution" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> Solution<span class="fold-unfold fa fa-minus-square"></span></button></div>
<ol>
<li><code style="color: inherit">countries</code>, <code style="color: inherit">olympics</code>, <code style="color: inherit">olympics_2022</code></li>
<li>Each are tables with 10 or more columns.</li>
</ol>
</details>
</blockquote>
<h2 id="about-this-dataset">About this dataset</h2>
<p>The data was <a href="https://github.com/UOSCS/Olympic_Athletes">obtained</a> from <a href="https://www.olympedia.org/">Olympedia</a>. The <code style="color: inherit">olympics</code> table contains
234,522 rows and 17 columns. Each row corresponds to an individual athlete competing in an individual Olympic event. The columns are:</p>
<ul>
<li><strong>athlete_id</strong> - Unique number for each athlete</li>
<li><strong>name</strong> - Athlete’s name</li>
<li><strong>sex</strong> - M or F</li>
<li><strong>birth_year</strong> - 4-digit number</li>
<li><strong>birth_day</strong> - e.g. 24 July</li>
<li><strong>birth_place</strong> - town and/or country</li>
<li><strong>height</strong> - In centimeters (or <code style="color: inherit">NA</code> if data not known)</li>
<li><strong>weight</strong> - In kilograms (or <code style="color: inherit">NA</code> if data not known)</li>
<li><strong>team</strong> - Team name</li>
<li><strong>noc</strong> - National Olympic Committee 3-letter code</li>
<li><strong>games</strong> - Year and season</li>
<li><strong>year</strong> - Integer</li>
<li><strong>season</strong> - Summer or Winter</li>
<li><strong>city</strong> - Host city</li>
<li><strong>sport</strong> - Sport</li>
<li><strong>event</strong> - Event</li>
<li><strong>medal</strong> - Gold, Silver, Bronze (or <code style="color: inherit">NA</code> if no medal was won)</li>
</ul>
<p>We will use this dataset to practice our data manipulation skills in Galaxy.</p>
<h1 id="choose-your-adventure">Choose your adventure!</h1>
<p>This tutorial is structured a bit differently than most. <strong>You do not have to do the steps in the order they are presented below.</strong> Every section in this tutorial uses the dataset you just uploaded (the <code style="color: inherit">olympics.db</code> file) as input, so you can jump to any section in this tutorial right now if you have a particular data manipulation operation in mind you want to learn more about.</p>
<h1 id="sorting">Sorting</h1>
<p>We have a lot of data in this file, but it is ordered by the athlete ID number, which is a somewhat arbitrary and meaningless number. But we can sort the rows in this file to something more convenient, for example alphabetically by name of the athlete, or chronologically by year of the Olympics.</p>
<p>In <code style="color: inherit">SQL</code> we can use the <code style="color: inherit">ORDER BY</code> clause. We’ll start by limiting our results, as every table in this dataset is quite large.</p>


In [ ]:
%%sql
SELECT NOC, `CLDR display name` FROM countries LIMIT 30;

<p>You can use <code style="color: inherit">ORDER BY column-name ASC</code> or <code style="color: inherit">ORDER BY column-name DESC</code> to sort the data ascending or descending.</p>


In [ ]:
%%sql
SELECT NOC, `CLDR display name` FROM countries ORDER BY NOC ASC LIMIT 30;

In [ ]:
%%sql
SELECT NOC, `CLDR display name` FROM countries ORDER BY NOC DESC LIMIT 30;

<p>So let’s sort our file in chronological order, based on the year of the Olympic games:</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-1"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<ol>
<li>Which column contains the year?</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-1"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-1" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> Solution<span class="fold-unfold fa fa-minus-square"></span></button></div>
<ol>
<li><code style="color: inherit">year</code></li>
</ol>
</details>
</blockquote>


In [ ]:
%%sql
SELECT * FROM olympics ORDER BY year LIMIT 30;

<p>If we wanted to do it in reverse, we could just use <code class="language-plaintext highlighter-rouge">order by year desc</code></p>


In [ ]:
%%sql
SELECT * FROM olympics ORDER BY year DESC LIMIT 30;

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-2"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<ol>
<li>Write a query to access only the first entry.</li>
<li>Which athlete is listed at the top of the file now?</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-2"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-2" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> Solution<span class="fold-unfold fa fa-minus-square"></span></button></div>
<ol>
<li>We can use <code style="color: inherit">LIMIT</code> for this.
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">select * from olympics order by year limit 1;
</code></pre></div>        </div>
</li>
<li>J. Defert. Who competed in a Tennis event 1896 Summer Olympics in Athens.</li>
</ol>
</details>
</blockquote>
<p>This is great, but maybe it would make more sense to sort alphabetically by athlete name <em>within each year</em>.</p>
<h2 id="sort-on-multiple-columns-at-once">Sort on multiple columns at once</h2>
<p>So we want to sort twice, first by year, an then within each year, we sort again alphabetically by name.</p>
<p>We will sort the file in chronological order based on the year of the Olympic games</p>


In [ ]:
%%sql
SELECT * FROM olympics ORDER BY year, name LIMIT 30;

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-3"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<p>Which athlete is listed at the top now? Which discipline (sport) did they compete in?</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-3"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-3" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> Solution<span class="fold-unfold fa fa-minus-square"></span></button></div>
<ol>
<li>A. Grigoriadis. He competed in the 500 meters freestyle swimming event.</li>
</ol>
</details>
</blockquote>
<h2 id="exercises">Exercises</h2>
<p>Ok, time to train! Let’s see if you can use the sort tool to answer the following questions:</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-exercise-reverse-the-sort"><i class="far fa-question-circle" aria-hidden="true"></i> Exercise: Reverse the sort</div>
<p>Which athlete comes <em>last by alphabet</em>, in the <em>most recent</em> Olympics?</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-4"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-4" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> Solution<span class="fold-unfold fa fa-minus-square"></span></button></div>
<p><code style="color: inherit">Žolt Peto</code> who competed in table tennis at the 2020 Summer Olympics in Tokyo.</p>
<p>We do this by repeating the previous sort (on year and then name), but changing the order to <em>descending</em> for both, to get the answer to the top of the file.</p>
</blockquote>
</blockquote>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-exercise-sort-by-height"><i class="far fa-question-circle" aria-hidden="true"></i> Exercise: sort by height</div>
<ol>
<li>What is the height of the tallest competing athlete? Which athlete(s) are of this height?</li>
<li>What is the shortest?</li>
<li>Who was the tallest athlete from the most recent Olympics? How tall were they?</li>
</ol>
<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<div class="box-title tip-title" id="tip-removing-null-values"><button class="gtn-boxify-button tip" type="button" aria-controls="tip-removing-null-values" aria-expanded="true"><i class="far fa-lightbulb" aria-hidden="true" ></i> Tip: Removing null values<span class="fold-unfold fa fa-minus-square"></span></button></div>
<p>This will be covered more during the Filtering section, but for now simply use this filter:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT * from olympics where height is not null ... ;
</code></pre></div>    </div>
</blockquote>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-hints"></div>
<ol>
<li>We can use <code style="color: inherit">.height</code>, and because we want the tallest on top, we will need to sort in <em>descending</em> (decreasing) order. Unfortunately you might discover there are null values.</li>
<li>Rerun the same query as step 1, but change the order to <em>ascending</em></li>
<li>First sort by year (descending), then by height (descending)</li>
</ol>
</blockquote>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-answers"></div>
<ol>
<li>Adam Sandurski from Poland is the tallest athlete in the file, at 214 cm tall.</li>
<li>Lyton Mphande from Seol is the shortest at 127 cm.</li>
<li>Gennaro Di Mauro, 210 cm. (2020 Summer Olympics in Tokyo)</li>
</ol>
</blockquote>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-full-solutions"></div>
<ol>
<li><code style="color: inherit">select * from olympics  where height is not null order by height desc limit 1;</code></li>
<li><code style="color: inherit">select * from olympics  where height is not null order by height asc limit 1</code></li>
<li><code style="color: inherit">select * from olympics  where height is not null order by year desc, height desc limit 1</code></li>
</ol>
</blockquote>
</blockquote>
<h1 id="filtering">Filtering</h1>
<p>This file contains a lot of data, but we may only be interested in a subset of this data. For example, we may only want to look at one particular Olympics, or one particular sport. In such cases we can filter the dataset. This will create a new dataset, removing any rows that are not of interest to us (i.e. that don’t meet the criteria we provide).</p>
<p>We will filter the file to show only winter Olympics
Look at the <code style="color: inherit">olympics</code> table and answer the following questions</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-4"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<ol>
<li>Which key contains this information?</li>
<li>Which values can this column have? (make sure to notice capitalisation, ‘Winter’ is not the same as ‘winter’ to these tools)</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-5"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-5" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> Solution<span class="fold-unfold fa fa-minus-square"></span></button></div>
<ol>
<li><code style="color: inherit">season</code></li>
<li>The values can be <code style="color: inherit">Summer</code> or <code style="color: inherit">Winter</code> (<code class="language-plaintext highlighter-rouge">select distinct season from olympics</code>)</li>
</ol>
</blockquote>
</blockquote>
<p>We’ll be using the <code style="color: inherit">WHERE</code> filter to select entries matching specific conditions:</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-5"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<ol>
<li>How would you write the expressions for the following conditions:
<ol>
<li><code style="color: inherit">enrolled</code> must be ‘Yes’</li>
<li><code style="color: inherit">age</code> must be smaller than 75</li>
<li><code style="color: inherit">height</code> cannot be null</li>
<li><code style="color: inherit">birthplace</code> cannot be empty</li>
</ol>
</li>
<li>It is also possible to combine multiple conditions, using <code style="color: inherit">and</code>, <code style="color: inherit">or</code>, <code style="color: inherit">not</code> and parentheses
How would you write expressions for the following filtering conditions:
<ol>
<li><code style="color: inherit">height</code> is larger than 200 or smaller than 160</li>
<li><code style="color: inherit">height</code> is larger than 200 and smaller than 210</li>
</ol>
</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-6"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-6" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> Solution<span class="fold-unfold fa fa-minus-square"></span></button></div>
<ol>
<li>The answers are:
<ol>
<li><code style="color: inherit">select * from olympics where enrolled = 'Yes'</code></li>
<li><code style="color: inherit">select * from olympics where age &lt; 75</code></li>
<li><code class="language-plaintext highlighter-rouge">select * from olympics where height is not null</code>`</li>
<li><code style="color: inherit">select * from olympics where birthplace != ""</code></li>
</ol>
</li>
<li>The answers are:
<ol>
<li><code style="color: inherit">select * from olympics where height &gt; 200 or height &lt; 160</code></li>
<li><code style="color: inherit">select * from olympics where height &gt; 200 and height &lt; 210</code></li>
</ol>
</li>
</ol>
</blockquote>
</blockquote>
<p>Ok, great, now that you’ve got the hang of writing expressions for this tool, let’s create a file with only Winter Olympics. Make sure it is contained in an array, in case we want to do further sorting.</p>


In [ ]:
%%sql
CREATE TABLE winter AS SELECT * FROM olympics WHERE season = 'Winter'

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-6"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<p>How many entries are in this file? (Hint: use <code style="color: inherit">count(*)</code>)</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-7"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-7" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> Solution<span class="fold-unfold fa fa-minus-square"></span></button></div>
<p>44,680</p>
</details>
</blockquote>
<p><strong>Repeat</strong> the step for the Summer Olympics</p>


In [ ]:
%%sql
CREATE TABLE summer AS SELECT * FROM olympics WHERE season = 'Summer'

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-7"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<ol>
<li>How many lines do you expect in the this file?</li>
<li>How many lines are in this file? Were you right?</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-hints-1"></div>
<ol>
<li>Use the <code style="color: inherit">count(*)</code> select</li>
<li>Be careful to consider whether these counts include the header line of the file or not</li>
</ol>
</details>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-answers-1"></div>
<ol>
<li>The original file has 234,522 entries, and the Winter Olympics had 44,680 entries. So we would expect 234,522 - 44,680 = 189,842 rows of data.
It is always useful to take a moment to think about the expected outcome, this makes it easier to spot mistakes and will save you time in the long run.</li>
</ol>
</blockquote>
</blockquote>
<h2 id="exercises">Exercises</h2>
<p>Ok, time to train! let’s see if you can use the <code style="color: inherit">select</code> filter to answer the following questions:</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-exercise-medal-winners"><i class="far fa-question-circle" aria-hidden="true"></i> Exercise: Medal winners</div>
<ol>
<li>How many gold medals were handed out?</li>
<li>How many total medals?</li>
<li>How many medals were handed out during the 2018 Olympics?</li>
<li>How many medals were won by individuals with a height between 170 and 180 cm? (inclusive)</li>
<li>How many gold medals were won by individuals shorter than 160cm or taller than 190?</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-hints-2"></div>
<ul>
<li>Column 17 contains information about medals</li>
<li>The possible values are <code style="color: inherit">Gold</code>, <code style="color: inherit">Silver</code>, <code style="color: inherit">Bronze</code>, and <code style="color: inherit"></code> (empty).</li>
<li>Don’t forget that the output (and line count) may include the header line</li>
<li>Do not use quotes on number columns (e.g. year)</li>
<li>You may need parentheses for complex conditions</li>
</ul>
</blockquote>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-answers-2"></div>
<ol>
<li>8,110   (Expression: <code style="color: inherit">SELECT count(*) FROM olympics WHERE medal == "Gold"</code>)</li>
<li>24,633  (Expression: <code style="color: inherit">SELECT count(*) FROM olympics WHERE medal == "Gold" or medal == "Silver" or medal == "Bronze")</code>, or <code style="color: inherit">medal != null</code>)</li>
<li>131     (Expression: <code style="color: inherit">SELECT count(*) FROM olympics WHERE medal == "Gold" and year == 2018</code> (note: do not use quotes around <code style="color: inherit">2018</code>, as it is a numerical value))</li>
<li>8,086   (Expression: <code style="color: inherit">SELECT count(*) FROM olympics WHERE medal is not null and height &gt;= 170 and height &lt;=180</code>)</li>
<li>2,333   (Expression: <code style="color: inherit">SELECT count(*) FROM olympics WHERE medal is not null and (height &lt; 160 or height &gt; 190)</code> (note: parentheses are important here))</li>
</ol>
<p>Note: these numbers are found by determining the number of lines in the file after each filtering step, and subtracting 1 for the header line.</p>
</blockquote>
</blockquote>
<h1 id="counting">Counting</h1>
<p>A common operation we might want to perform on tables of data, is simple counting. How many times does a certain value appear? For our dataset for instance, we might want to know how many countries participated in each Olympics, how many women, etc; any column that has categorical data that we can count.</p>
<p>Let’s start by simply counting how many different Olympic Games we have in our dataset, and how many times it appears (so how many participations there were each year)</p>
<p>We’ll need to use the <code style="color: inherit">group by</code> syntax which takes a key, and then groups by those values.</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-8"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<ol>
<li>How many different Olympic games are in our file?</li>
<li>Which Olympic games had the most participations? (Tip: use order by)</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-8"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-8" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> Solution<span class="fold-unfold fa fa-minus-square"></span></button></div>
<ol>
<li>
<p>52 games (<code class="language-plaintext highlighter-rouge">select count(*), games from olympics group by games</code>)</p>
<p>The resulting file looks something like:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"> 615	1896 Summer Olympics
 2503	1900 Summer Olympics
 2643	1904 Summer Olympics
 3213	1908 Summer Olympics
 4610	1912 Summer Olympics
 3448	1920 Summer Olympics
 5242	1924 Summer Olympics
 358	1924 Winter Olympics
 4493	1928 Summer Olympics
 ...
</code></pre></div>        </div>
</li>
<li>
<p>1996 Summer Olympics. (10501 participations)</p>
</li>
</ol>
</blockquote>
</blockquote>
<p>You may have guessed that like <code style="color: inherit">order by</code>, that we could have selected multiple columns in the <code style="color: inherit">group by</code> step. This lets us count on combinations of columns.</p>
<p>Let’s try counting the number of men and women in each olympic games.</p>


In [ ]:
%%sql
select count(*), games, sex from olympics group by games, sex

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-9"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<p>You see the resulting file has a line for every combination of the two columns (games and sex), providing the count for each.</p>
<ol>
<li>
<p>How many women were in the first Olympic games?</p>
</li>
<li>
<p>Which Olympic games had the most women participants?</p>
</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-9"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-9" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> Solution<span class="fold-unfold fa fa-minus-square"></span></button></div>
<ol>
<li>2 women participated in the 1896 Olympics. (note that we cannot be sure if this is two different women, or 1 woman participating twice, in this query. Do you know any way we could query that? Try it out!)
The results looks something like this:
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">2	F	1896 Summer Olympics
43	F	1900 Summer Olympics
17	F	1904 Summer Olympics
55	F	1908 Summer Olympics
97	F	1912 Summer Olympics
132	F	1920 Summer Olympics
269	F	1924 Summer Olympics
</code></pre></div>        </div>
</li>
<li>2020 Summer Olympics (4652)</li>
</ol>
</details>
</blockquote>
<p>Let’s say we wanted to know how many different sports there were in each Olympics. If we used the counting query above, we would get resultsfor each combination of sport and olympics, with the number of lines (participations) of each. But we don’t really care about the number of lines that have this combination, just the total number of unique sports in each games.</p>
<p>We can use the <code style="color: inherit">distinct</code> filter in our pipeline to discover this. First let’s do our group by and iterate over each resulting group:</p>


In [ ]:
%%sql
select games, sport from olympics group by games;

<p>And let’s count all of their appearances</p>


In [ ]:
%%sql
select games, count(sport) as sports from olympics group by games;

<p>But those results still aren’t distinct, those numbers are far too high. So let’s use distinct:</p>


In [ ]:
%%sql
select games, count(distinct sport) as sports from olympics group by games;

<p>We’re almost there! Let’s sort this</p>


In [ ]:
%%sql
select games, count(distinct sport) as sports from olympics group by games order by sports asc;

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-10"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<ol>
<li>How many sport were in the first Olympics? How many in the latest?</li>
<li>Which Olympics had the most different sports?</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-10"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-10" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> Solution<span class="fold-unfold fa fa-minus-square"></span></button></div>
<ol>
<li>10 and 38.</li>
<li>The 2020 Summer Olympics had the most different sports (38)</li>
</ol>
</details>
</blockquote>
<p>Save the output as something descriptive.</p>
<h2 id="exercises">Exercises</h2>
<p>Ok, let’s practice!</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-exercise-number-of-participations-per-country"><i class="far fa-question-circle" aria-hidden="true"></i> Exercise: Number of participations per country</div>
<ol>
<li>Which country has had the most participations in the Olympics?</li>
<li>How many countries participated in the first Olympics? How many in the last?</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-hints-3"></div>
<ol>
<li>Since we are counting instances of a key, we can use <code style="color: inherit">group by team</code> and then loop over that to print out the length, and the team name of each of those items.</li>
<li>This is basically the same question as “how many women” participated, try modifying that query.</li>
</ol>
</blockquote>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-answers-3"></div>
<ol>
<li>The United States with 17,286 participations (<code class="language-plaintext highlighter-rouge">select team, count(team) as count from olympics group by team order by count desc;</code>)</li>
<li>15 and 250. (<code class="language-plaintext highlighter-rouge">select games, count(distinct team) as teams from olympics group by games;</code>)</li>
</ol>
</blockquote>
</blockquote>
<h1 id="grouping">Grouping</h1>
<p>Often we may want to group rows based on a value in a column, and perform some operation on the resulting rows. For example we would like to group the olympics data by one value (e.g. year, country, sport), and determine some value for each group (e.g. number of medals won, average age of athletes, etc).</p>
<p>In the <a href="#counting">counting</a> section of this tutorial we show how to get answers that require a count (e.g. number of medals won), but sometimes we want to do something more complex, like calculating the average height of athletes in a group, say per country or per sport. This section will show some example of these types of questions.</p>
<p>We can use continue to use group by for this, but now we’ll need the max and min aggregate operations. Essentially every time we use <code style="color: inherit">group by</code> we need to use an aggregation like finding the maximum, minimum, or counting the number of results.</p>
<blockquote class="hands_on" style="border: 2px solid #dfe5f9; margin: 1em 0.2em">
<div class="box-title hands-on-title" id="hands-on-tallest-athlete-per-sport"><i class="fas fa-pencil-alt" aria-hidden="true" ></i> Hands-on: Tallest athlete per sport</div>
<p>We would like to answer the following question: <em>How tall was the tallest athlete of each sport?</em></p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-11"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<ol>
<li>How tall was the tallest athlete in basketball? And what about karate?</li>
<li>Why do some sports have null values?</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-11"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-11" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> Solution<span class="fold-unfold fa fa-minus-square"></span></button></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">select max(height), min(height),sport from olympics group by sport
</code></pre></div>      </div>
<ol>
<li>Basketball’s tallest athlete was 192cm. For Karate it is 163.</li>
<li>Our dataset had quite a number of <code style="color: inherit">null</code> (unknown) values in the height column, especially for the earlier Olympics. These are preserved in the outputs.</li>
</ol>
</blockquote>
</blockquote>
</blockquote>
<h2 id="grouping-on-multiple-columns">Grouping on multiple columns</h2>
<p>You may have noticed that we could also provide multiple columns to group on. If we do this, we can compute values for combinations of groups, such as sex and sport, to find e.g. the tallest woman in basketball or the shortest man per Olympics. There are also many more options for the computation we perform, so perhaps we are more interested not in the tallest athlete, but the average height. Let’s perform some of these slightly more advanced queries now.</p>
<blockquote class="hands_on" style="border: 2px solid #dfe5f9; margin: 1em 0.2em">
<div class="box-title hands-on-title" id="hands-on-average-height-of-men-and-women-per-sport"><i class="fas fa-pencil-alt" aria-hidden="true" ></i> Hands-on: Average height of men and women per sport</div>
<p>The question we would like to answer here, is what is the average height for men and women per sport?</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">select avg(height), sex, sport from olympics group by sex, sport;
</code></pre></div>  </div>
<p>See if you can answer the following questions based on the output file.</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-12"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<ol>
<li>What is the average height of women participating in archery?</li>
<li>What is the average height of men participating in <a href="https://en.wikipedia.org/wiki/Ballooning_at_the_1900_Summer_Olympics">ballooning</a>?</li>
<li>Why do some values have <code style="color: inherit">null</code> instead of a height?</li>
<li>Why do some sports not have a value for one of the sexes?</li>
<li>Can you find a sport where women were taller than the men? (Hint: it starts with the letter A)</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-12"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-12" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> Solution<span class="fold-unfold fa fa-minus-square"></span></button></div>
<ol>
<li>167.25677031093 cm</li>
<li>170 cm</li>
<li>If none of the rows in the group had height data available, it will output <code style="color: inherit">nan</code> (not a number) instead. This is most common for sports that were only featured a few times in the early years of the Olympics.</li>
<li>Sports such as artistic swimming only exist for women, so no M appears in the data for that group, so there simply is no row for the mean height of men doing artistic swimming in our output.</li>
<li><a href="https://en.wikipedia.org/wiki/Art_competitions_at_the_Summer_Olympics">Art Competitions</a></li>
</ol>
<p>If all went well, your output file should look something like:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">GroupBy(sport)	     GroupBy(sex)  mean(height)
Aeronautics         M             nan
Alpine Skiing       F             167.38324708926
Alpine Skiing       M             178.18747142204
Alpinism            M             nan
Archery             F             167.25677031093
Archery             M             178.5865470852
Art Competitions    F             175.33333333333
Art Competitions    M             173.97260273973
Artistic Gymnastics F             156.15316901408
</code></pre></div>      </div>
</blockquote>
</blockquote>
</blockquote>
<h2 id="exercises-1">Exercises</h2>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-exercise-grouping-and-computing"><i class="far fa-question-circle" aria-hidden="true"></i> Exercise: Grouping and computing</div>
<ol>
<li>How tall is the shortest woman Badminton player to win a gold medal?</li>
<li>What is the average height of athletes from team Denmark in the 1964 Olympics? (Note: 1964 has summer and winter olympics)</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-hints-4"></div>
<ol>
<li>We need to group on 3 columns: medal, sport and sex, and then select the <code style="color: inherit">min</code>.</li>
<li>We need to group on 2 columns: country (team) and year, then compute the average over height.</li>
</ol>
</blockquote>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-answers-4"></div>
<ol>
<li>161 cm.</li>
<li>mean height: 175.91304347826, standard deviation: 7.0335410308672`</li>
</ol>
</blockquote>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-full-solutions-1"></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">select min(height), medal, sport, sex from olympics group by medal, sport, sex;
select avg(height), team, games from olympics where team = "Denmark" group by team, games;
</code></pre></div>    </div>
</blockquote>
</blockquote>
<h1 id="computing">Computing</h1>
<p>Sometimes we want to use the data in our column to compute a new value, and add that to the table. For instance, for our dataset we could caluclate athtletes BMI (using height and weight columns), or their age at time of participation (from year of birth and year of the Olymics). By adding these computed values as a new colum to our datset, we can more easily query the dataset for these values. We can do these types of operations on the fly, and then if we like, store them as (temporary) tables.</p>
<p>As an example, let’s calculate the age of each athlete at the time of participation, and add this as a new column to our dataset.</p>


In [ ]:
%%sql
select year - birth_year as age, games from olympics LIMIT 30;

<p>If we want to save that result to make it easier to query, then we have a couple options.</p>
<ol>
<li>
<p>Don’t store it, calculate on demand. Very storage efficient.</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">select noc, name, ..., year - birth_year as age from olympics
</code></pre></div>    </div>
</li>
<li>
<p>Create a temporary table with this data, great if we only need it temporarily</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">create temporary table olympics_ages as select *, year - birth_year as age from olympics
</code></pre></div>    </div>
</li>
<li>
<p>Create a new permanent table with this data</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">create table olympics_ages as select *, year - birth_year as age from olympics
</code></pre></div>    </div>
</li>
<li>
<p>Update the existing table by adding it as a new column</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">alter table olympics add column age int;
update olympics set age = year - birth_year;
</code></pre></div>    </div>
</li>
</ol>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-13"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<ol>
<li>How old was Arnaud Boetsch during his Olympic tennis participation?</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-13"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-13" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> Solution<span class="fold-unfold fa fa-minus-square"></span></button></div>
<ol>
<li>Arnaud Boetsch is listed on the first two lines, who turned 27 the year of their Olympics.</li>
</ol>
</details>
</blockquote>
<p>This was a simple computation, but much more complex mathematical expressions can be computed with this tool. In the exercise below, we will compute the BMI for each athlete as an example.</p>
<h2 id="exercises">Exercises</h2>
<p>BMI stands for Body Mass Index, is a metric to provide a very crude measure of how healthy your weight is. The formula to compute BMI is:</p>

\[BMI = weight / (height^2)\]
<p>(with weight in kilograms and height in meters).</p>
<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<div class="box-title tip-title" id="tip-bmi-is-problematic"><button class="gtn-boxify-button tip" type="button" aria-controls="tip-bmi-is-problematic" aria-expanded="true"><i class="far fa-lightbulb" aria-hidden="true" ></i> Tip: BMI is problematic<span class="fold-unfold fa fa-minus-square"></span></button></div>
<p><a href="https://en.wikipedia.org/wiki/Adolphe_Quetelet">Adolphe Quetelet</a>, who invented <a href="https://en.wikipedia.org/wiki/Body_mass_index">BMI</a>, was not a doctor of medicine, instead he was a Belgian astronomer, mathematician, statistician, and sociologist. His data consisted probably entirely of cisgender, white, european men and women {% cite Eknoyan_2007 %}.
In his defense, he did argue it should not be used at individual levels, however it came to be used as such due to simplicity. (See <a href="https://en.wikipedia.org/wiki/Body_mass_index#History">Wikipedia’s History section</a>). BMI is a poor measure of health, especially for populations with high muscle content, and cannot be simply re-used as-is with anyone other than Anglo Saxons {% cite Caleyachetty_2021 %}, <a href="https://en.wikipedia.org/wiki/Body_mass_index#Limitations">Wikipedia:Limitations</a>.</p>
<p>However as it is an easy to calculate metric, we include the calculation here.</p>
</blockquote>
<p>Let’s compute this data for all athletes and add it as a new column!</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-exercise-calculating-bmi"><i class="far fa-question-circle" aria-hidden="true"></i> Exercise: Calculating BMI</div>
<ol>
<li>How would you express this calculation in SQL?
<ul>
<li>Remember that our height is in cm, and the formula expects height in meters</li>
<li>And that we have null values, but we can ignore those as they will make the final calculation null as well</li>
</ul>
</li>
<li>What is the BMI for Arnaud Boetsch?</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-hints-5"></div>
<ul>
<li>division is <code style="color: inherit">/</code> and multiplication is <code style="color: inherit"> * </code> .</li>
<li>Generally we cannot use <code style="color: inherit">^</code></li>
<li>Parentheses may be required.</li>
<li>use <code style="color: inherit">select(.value != null)</code> to remove nulls.</li>
<li>If that isn’t eough, you can check that the type is a number with <code style="color: inherit">(.value|type) == "number"</code> to ensure it really is a number and not e.g. a string.</li>
<li>remember to wrap everything in <code style="color: inherit">[...]</code> to retain the data shape</li>
</ul>
</blockquote>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-answers-5"></div>
<ol>
<li>
<p>other variations are possible:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">create temporary table olympics_bmi as select *, weight / (height / 100 * height / 100) as bmi from olympics;
</code></pre></div>        </div>
</li>
<li>
<p>22.69</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">select * from olympics_bmi where name like 'Arnaud Boetsch';
</code></pre></div>        </div>
</li>
</ol>
</blockquote>
</blockquote>
<h1 id="find-and-replace">Find and Replace</h1>
<p>Often you may need to change the contents of a file a bit to fit the expectations of an analysis tool. For instance, our database uses <code style="color: inherit">null</code> for missing values, but other conventions included leaving the cell empty instead. Or, when working with chromosomal data, you may need to add or remove the <code style="color: inherit">chr</code> prefix from a column before using it as input to a certain tool. In such situations, we can find all occurrences of a certain pattern in our file, and replace it with another value.</p>
<p>If we want to perform such a replacement on a single column in our data, we can use an update statement.</p>
<p>A few of the basics of regular expression, plus some links to further resources are given in the box below:</p>
<p>Let’s start with a simple example:
Our file uses a mix of <code style="color: inherit">Athina</code> and <code style="color: inherit">Athens</code> to indicate the Capital City of Greece in the <code style="color: inherit">city</code> column.
Let’s standardize this by replacing occurrences of <code style="color: inherit">Athina</code> with <code style="color: inherit">Athens</code>.</p>
<p>Let’s start by filtering out the old spelling:</p>


In [ ]:
%%sql
select * from olympics where city = 'Athina' limit 30;

<p>Let’s try replacing it:</p>


In [ ]:
%%sql
update olympics
set city = 'Athens'
where city = 'Athina';

<p>Look at the file before and after. Athlete 7 (Patrick Chila) near the top of the <code style="color: inherit">olympics.tsv</code> file, had a value of Athina in the city column. Verify that it has been changed to Athens.</p>
<p>This was rather simple example, so let’s try a few more examples with slightly more complex expressions.</p>
<h2 id="exercises">Exercises</h2>
<p>You may have noticed that our file has a lot of missing data. Especially for the earlier years, things like height, weight and birthday of athletes was not registered, or simply not known. In some columns you see these missing values have been replaced with an <code style="color: inherit">NA</code> (not available) value. In other columns (for example birth place), the cells have simply been left empty.</p>
<p>Different tools may expect different ways of handling missing data. So you may have to change your missing data from empty to <code style="color: inherit">NA</code>, <code style="color: inherit">NaN</code>, or something else, between analysis steps</p>
<blockquote class="hands_on" style="border: 2px solid #dfe5f9; margin: 1em 0.2em">
<div class="box-title hands-on-title" id="hands-on-fill-empty-cells"><i class="fas fa-pencil-alt" aria-hidden="true" ></i> Hands-on: Fill empty cells</div>
<p>We will now replace empty cells in the <code style="color: inherit">birth_place</code> column, to use <code style="color: inherit">null</code> instead.</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-hints-6"></div>
<p>Remember that comparison to nulls is done with <code style="color: inherit">is</code> instead of <code style="color: inherit">=</code></p>
</details>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-answers-6"></div>
<p>other variations are possible:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">update olympics set birth_place = null where birth_place = '';
</code></pre></div>    </div>
</blockquote>
</blockquote>
<p>Let’s do another example, this one splitting and re-constructing strings.</p>
<p>Look at the <code style="color: inherit">birth_day</code> column. It has values in a format like <code style="color: inherit">12 December</code>. Suppose we have a tool that expects this data to be in the reverse format, <code style="color: inherit">December 12</code>. We would not want to do this manually, but with sql we can make this replacement easily</p>
<p>We will now change the format in birthday column from <code style="color: inherit">day month</code> to <code style="color: inherit">month day</code>, as our boss is American and requested the silly format.</p>
<p>First we need to understand that sqlite does not ship a regex engine, thus we cannot use familiar regular expressions. Instead we can make use of <code style="color: inherit">instr(string, search)</code> to find the location of a substring like <code style="color: inherit"> </code>, identifying where the day stops and the month starts. Then we can use <code style="color: inherit">substr(string, start)</code> and <code style="color: inherit">substr(string, start, end)</code> to chop up our date string.</p>
<p>In SQL, concatenation is done with <code style="color: inherit">||</code>.</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-14"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<ol>
<li>How do we captures both the day and the month?</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-hints-7"></div>
<ol>
<li>We should use something like <code style="color: inherit">substr(birth_day, instr(birth_day, ' '))</code></li>
</ol>
</blockquote>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-answers-7"></div>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">select
  birth_day,
  instr(birth_day, ' ') as idx,
  substr(birth_day, instr(birth_day, ' ')) as month,
  substr(birth_day, 0, instr(birth_day, ' ')) as day
from olympics
limit 10
</code></pre></div>    </div>
<p>We can make our final query:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">select
  substr(birth_day, instr(birth_day, ' ')) || substr(birth_day, 0, instr(birth_day, ' ')) as birth_day_new
from olympics
limit 10
</code></pre></div>    </div>
<p>And then store this as a new column (using <code style="color: inherit">update</code>) or as a new table (using <code style="color: inherit">create [temporary] table</code>)</p>
</blockquote>
</blockquote>
<h1 id="removing-columns">Removing Columns</h1>
<p>In sqlite you cannot remove columns, it is not supported. Proper databases like Postgres and MySQL support this operation.
Instead for sqlite you could select the columns you want to keep, create a new table from that, and then delete the original table.</p>
<p>Other databases:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">alter table delete column NAME from olympics
</code></pre></div></div>
<h1 id="unique">Unique</h1>
<p>Sometimes, in the course of our data manipulations, we may end up with a file that has duplicate values. In order to filter these out, we can use the <code style="color: inherit">distinct</code> filter.</p>
<p>Let’s say we would like to create a list of all unique athletes (id and name).</p>
<p>First we will just select the <code style="color: inherit">athlete_id</code> and <code style="color: inherit">name</code> columns from our dataset</p>


In [ ]:
%%sql
select name, athlete_id from olympics LIMIT 30

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-15"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<ol>
<li>Do you see duplication? Why is that?</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-14"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-14" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> Solution<span class="fold-unfold fa fa-minus-square"></span></button></div>
<ol>
<li>Yes. For all athletes who participated more than once, the row will be identical.</li>
</ol>
</details>
</blockquote>
<p>Now let’s remove those duplicates.</p>


In [ ]:
%%sql
select distinct name, athlete_id from olympics limit 30

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-16"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<p>How many unique athletes do we have? Note that you cannot count multiple columns, so choose one that is correct.</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-15"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-15" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> Solution<span class="fold-unfold fa fa-minus-square"></span></button></div>
<p>94,733</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">select count(distinct athlete_id) from olympics;
</code></pre></div>    </div>
</details>
</blockquote>
<h1 id="joining-datasets">Joining Datasets</h1>
<p>This database contains a lot of information, but we may want to add more information. For example if we had a file with information about each country (population, capital city, etc), we could join that information with our Olympics data, to get a single result with all information in every row.</p>
<p>For example, if we would like to be able to group by continent, to e.g. count athletes, medals etc per continent, we will have to add a <code style="color: inherit">continent</code> column to our file. To do this we would need a second file that maps each country to the corresponding continent. This is what we will do in the next hands-on section.</p>
<p>We obtained country information data from <a href="https://datahub.io/core/country-codes">DataHub</a>. More information about this file can be found in the description there. It has 56 columns with a wide variety of data about each country (from country codes, to capital city, languages spoken, etc)</p>
<p>It is available in the countries table.</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-17"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<ol>
<li>Which keys(s) in this file are the same as in the <code style="color: inherit">olympics.tsv</code> file?</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-16"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-16" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> Solution<span class="fold-unfold fa fa-minus-square"></span></button></div>
<ol>
<li>Both files have a <code style="color: inherit">NOC</code> column with the 3-letter country code (<code style="color: inherit">NOC</code> stands for National Olympic Committee). However, one is lowercase.</li>
</ol>
</blockquote>
</blockquote>
<p>We would now like to take our Olympics dataset as the basis, and add columns to every row of this file with some information about the country. In order to join, we will need to have one column that is shared between the two files, on which we can match. The <code style="color: inherit">NOC</code> column is perfect for this because it is a defined standard. Both files also contain a column with the country name in it, which is also a possible candidate to use for joining, but because it is less standardised, it is safer to use the NOC column. For example, if one file uses “Netherlands”, while the other uses “The Netherlands” to indicate the same country, the joining will fail for these rows. So always make sure the columns you join on are compatible!</p>
<p>We can use the join commands.</p>


In [ ]:
%%sql
select * from olympics left join countries on olympics.noc = countries.NOC LIMIT 30

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-18"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<ol>
<li>What do you expect the output to look like? Were you right?</li>
<li>How many columns are in the resulting file? What about the NOC column?</li>
<li>What is a possible downside to this approach?</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-17"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-17" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> Solution<span class="fold-unfold fa fa-minus-square"></span></button></div>
<ol>
<li>All the columns from the country information file are added to the end of each row of our olympics dataset</li>
<li>Our olympics datset had 17 columns, the country information file has 56 columns. Therefore we have 17+56=73 columns columns in our resulting file. This also means the NOC column
we joined on appears twice in our output.</li>
<li>There is a lot of data duplication in the output now. The exact same country information is added to every line of every athlete from a certain country.
This means much larger response size.
If you do not need all these columns, it could save you a lot of space to select only specific columns that you require.</li>
</ol>
</details>
</blockquote>
<h1 id="concatenating">Concatenating</h1>
<p>Concatenation of two files simple means putting the contents of the two files together, one after the other. Our dataset was created in 2021, but since then we’ve had another Olympic event, the 2022 Winter Olympics in Beijing. If we have the same data for this latest Olympics, we could simply add the rows from the 2022 games to our current file with data, in order to create a single file with all data from 1896 to 2022.</p>
<p>View the table <code style="color: inherit">olympics_2022</code>, does it have the same structure as our original <code style="color: inherit">olympics</code> table?</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-19"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<ol>
<li>Does the new table have the same structure?</li>
<li>Can we simply add the lines of the new table to the end of our existing olympics dataset?</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-18"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-18" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> Solution<span class="fold-unfold fa fa-minus-square"></span></button></div>
<ol>
<li>Yes, this file has all the same columns, in the same order, so concatenation should be relatively straightforward.</li>
<li>Yes.</li>
</ol>
</blockquote>
</blockquote>
<p>Since this new dataset has the exact same structure (number and order of columns), we can simple add the lines from this file to the end of our existing <code style="color: inherit">olympic</code> table.
For this, we’ll need to use the <code style="color: inherit">union all</code> which takes two separate sql queries and unifies the results.</p>


In [ ]:
%%sql
select * from olympics
union all
select * from olympics_2022
LIMIT 30;

<p>(Note: We are limiting the outputs to ensure your browser does not crash loading all of the data.)</p>
<p>Now this only works so simply because our two datasets had the same structure. If your data comes from different sources, you may have to do some additional data manipulation before you can union, e.g. to make sure the columns match, or how each file deals with missing data (empty cells, <code style="color: inherit">NA</code>, <code style="color: inherit">NaN</code> or something else).</p>
<h1 id="conclusion">Conclusion</h1>
<p>These operations covered in the tutorial are just a few examples of some of the most common operations. There are many more available. We encourage you to look around the documentation of sqlite or your database. The more comfortable you are performing these kinds of steps, the more you can get out of SQL!</p>
<h1 id="exercises-putting-it-all-together">Exercises: Putting it all together!</h1>
<p>This section provides a number of exercises that require you to combine two or more of the techniques you learned in this tutorial. This is a great way to practice your data manipulation skills. Full solutions are provided for every exercise (i.e. all tools and settings), but for many of these exercises there will be multiple solutions, so if you obtained the same results in a different way, that is correct too!</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-exercise-1-finding-shortest-lightest-athlete"><i class="far fa-question-circle" aria-hidden="true"></i> Exercise 1: Finding shortest/lightest athlete</div>
<p>If you have done exercises in the <a href="#sorting">sorting</a> section, you noticed that finding the shortest athlete ever to compete was not easy,
because all the rows with missing height data (<code class="language-plaintext highlighter-rouge">NA</code>) in the column were sorted to the top. We need to filter out these values first, then
perform our sort, so that our answer is on top.</p>
<ol>
<li>Find the shortest athlete ever to compete in the Olympics</li>
<li>Find the shortest athlete of the Winter Olympics</li>
<li>Find the lightest athlete of the <em>most recent</em> Summer Olympics</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-hints-8"></div>
<ol>
<li>You will need to filter out the columns with (<code class="language-plaintext highlighter-rouge">NA</code>) in the height column first</li>
<li>You will need to filter by season as well</li>
<li>You will need to filter out missing data in the weight column, filter out Summer Olympics, then sort (by 2 columns)</li>
</ol>
</details>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-answers-8"></div>
<ol>
<li>Lyton Mphande and  Rosario Briones were both 127 cm tall, competing in boxing and gymnastics respectively</li>
<li>Carolyn Krau was a 137 cm tall figure skater.</li>
<li>Flávia Saraiva was the lightest athlete (31kg), she was a Artistic Gymnast from Brazil.</li>
</ol>
</blockquote>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-full-solution"></div>
<ol>
<li>
<p>First we filter out the NA values from the height column:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">select * from olympics where height is not null
</code></pre></div>        </div>
<p>Then we can sort by height, in ascending order to get the shortest athletes on top:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">select * from olympics where height is not null order by height desc limit 10;
</code></pre></div>        </div>
</li>
<li>
<p>We can take the output from the first exercise, and filter for only Winter Olympics:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">select * from olympics where height is not null and season = 'Winter' order by height desc limit 10;
</code></pre></div>        </div>
</li>
<li>
<p>First we filter out the NA values from the weight column:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">select * from olympics where weight is not null and games = '2020 Summer Olympics' order by weight asc limit 10;
</code></pre></div>        </div>
</li>
</ol>
</blockquote>
</blockquote>
<p>Congratulations! You have now mastered the basics of data manipulation! There are a lot more data manipulation operations available that you may need. Please explore the tools for yourself, and check back with this tutorial often, we plan to add more sections and exercises over time!</p>


# Key Points

- Basic data manipulation is often needed between steps in a larger scientific analysis in order to connect outputs from one tool to input of another.
- There are often multiple ways/tools to achieve the same end result
- Having a basic understanding of data manipulation tools will make it easier to do exploratory data analysis
- Always read the help text of the tool before using it to get a full understanding of its workings
- Always try to formulate the output you are expecting from a tool. This will make it easier to spot mistakes as soon as possible.

# Congratulations on successfully completing this tutorial!

Please [fill out the feedback on the GTN website](https://training.galaxyproject.org/training-material/topics/data-science/tutorials/data-manipulation-olympics-sql/tutorial.html#feedback) and check there for further resources!
